In [19]:
import sympy as sp
from typing import List
from sympy import sin, cos, pi, sqrt, acos, simplify, atan
import math
sp.init_printing()


def homogeneous(rotation: sp.Matrix = sp.eye(3), translation: sp.Matrix = sp.zeros(3, 1)) -> sp.Matrix:
    return rotation.row_join(translation).col_join(sp.Matrix([[0, 0, 0, 1]]))

def dh(rotation, twist, displacement, offset):
    rotation_mat = sp.Matrix([
        [cos(rotation), -sin(rotation)*cos(twist),  sin(rotation)*sin(twist)],
        [sin(rotation), cos(rotation)*cos(twist),   -cos(rotation)*sin(twist)],
        [0,             sin(twist),                 cos(twist)],
    ])
    translation = sp.Matrix([
        [offset*cos(rotation)],
        [offset*sin(rotation)],
        [displacement],
    ])
    return rotation_mat, translation

def rotation(homogeneous: sp.Matrix):
    return homogeneous[:3, :3]

def translation(homogeneous: sp.Matrix):
    return homogeneous[:3, 3:]

def jacobian(transforms: List[sp.Matrix], joint_types: List[sp.Matrix], base_z: sp.Matrix = sp.Matrix([0, 0, 1])):
    transforms_chained = [homogeneous()]
    z_unit_vecs = []
    # chain the transforms
    for i, transform in enumerate(transforms):
        transforms_chained.append(transforms_chained[-1] * transform)
    # apply rotation of chained transforms to base z
    for chained_transform in transforms_chained:
        z_unit_vecs.append(rotation(chained_transform) * base_z)
    
    assert len(transforms_chained) == len(z_unit_vecs)

    jacobian = sp.zeros(6, len(transforms))
    for i, (transform, joint_type) in enumerate(zip(transforms, joint_types)):
        if joint_type == 'revolute':
            jacobian[:3, i] = z_unit_vecs[i].cross(translation(transforms_chained[-1]) - translation(transforms_chained[i]))
            jacobian[3:, i] = z_unit_vecs[i]
        elif joint_type == 'prismatic':
            jacobian[:3, i] = z_unit_vecs[i]
            jacobian[3:, i] = sp.Matrix([[0], [0], [0]])

        # angular velocity

    return jacobian

def skew(v: sp.Matrix):
    return sp.Matrix([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0],
    ])

In [20]:
q1, q2, q3 = sp.symbols('q_1 q_2 q_3')
l0, l1, l2 = 1, 1, 1.5
k1, k2, k3 = 4e5, 2e5, 1e5
K = sp.diag(k1, k2, k3)
joint1 = homogeneous(*dh(q1, pi/2, 0, l0))
joint2 = homogeneous(*dh(q2, 0, l1, 0))
joint3 = homogeneous(*dh(q3, 0, l2, 0))

J = jacobian([joint1, joint2, joint3], ['revolute', 'revolute', 'revolute'])
C = J*K.inv()*J.T

display(simplify(J))
display(simplify(C))
display(simplify(C.evalf(subs={q1: math.radians(90), q2: math.radians(135), q3: math.radians(-90)})))

F = sp.Matrix([100, 50, -30, 0, 0, 0])

R = rotation(joint1 * joint2 * joint3)
p = -(joint1*joint2*joint3*sp.Matrix([0, 0, 0, 1]))
S = sp.zeros(6, 6)
S[:3, :3] = R.T
S[:3, 3:] = skew(p)*R.T
S[3:, 3:] = R.T
F = S*F
(C*F).evalf(subs={q1: math.radians(90), q2: math.radians(135), q3: math.radians(-90)})

⎡-sin(q₁) + 2.5⋅cos(q₁)     0         0    ⎤
⎢                                          ⎥
⎢2.5⋅sin(q₁) + cos(q₁)      0         0    ⎥
⎢                                          ⎥
⎢          0                0         0    ⎥
⎢                                          ⎥
⎢          0             sin(q₁)   sin(q₁) ⎥
⎢                                          ⎥
⎢          0             -cos(q₁)  -cos(q₁)⎥
⎢                                          ⎥
⎣          1                0         0    ⎦

⎡                                      2                                      
⎢1.5625e-5⋅(-0.4⋅sin(q₁) + 1.0⋅cos(q₁))   6.5625e-6⋅sin(2⋅q₁) + 6.25e-6⋅cos(2⋅
⎢                                                                             
⎢                                                                             
⎢6.5625e-6⋅sin(2⋅q₁) + 6.25e-6⋅cos(2⋅q₁)  1.5625e-5⋅(1.0⋅sin(q₁) + 0.4⋅cos(q₁)
⎢                                                                             
⎢                   0                                        0                
⎢                                                                             
⎢                                                                             
⎢                   0                                        0                
⎢                                                                             
⎢                                                                             
⎢                   0                               

⎡ 2.5e-6   -6.25e-6   0            0                      0            -2.5e-6
⎢                                                                             
⎢-6.25e-6  1.5625e-5  0            0                      0            6.25e-6
⎢                                                                             
⎢   0          0      0            0                      0               0   
⎢                                                                             
⎢   0          0      0         1.5e-5          -9.18485099360515e-22     0   
⎢                                                                             
⎢   0          0      0  -9.18485099360515e-22  5.62409918498197e-38      0   
⎢                                                                             
⎣-2.5e-6    6.25e-6   0            0                      0            2.5e-6 

⎤
⎥
⎥
⎥
⎥
⎥
⎥
⎥
⎥
⎥
⎦

⎡0.000388908729652601 ⎤
⎢                     ⎥
⎢-0.000972271824131503⎥
⎢                     ⎥
⎢          0          ⎥
⎢                     ⎥
⎢          0          ⎥
⎢                     ⎥
⎢          0          ⎥
⎢                     ⎥
⎣-0.000388908729652601⎦